# Attentional Networks in Computer Vision

Prepared by comp411 Teaching Unit (TA Can Küçüksözen) in the context of Computer Vision with Deep Learning Course. Do not hesitate to ask in case you have any questions, contact me at: ckucuksozen19@ku.edu.tr

Up until this point, we have worked with deep fully-connected networks, convolutional networks and recurrent networks using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, on the other hand, most successful image processing methods use convolutional networks. However recent state-of-the-art results on computer vision realm are acquired using Attentional layers and Transformer architectures.

First you will implement several layer types that are used in fully attentional networks. You will then use these layers to train an Attentional Image Classification network, specifically a smaller version of Vision Transformer (VIT) on the CIFAR-10 dataset. The original paper can be accessed via the following link: https://arxiv.org/pdf/2010.11929.pdf

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [3]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp411/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./comp411/datasets\cifar-10-python.tar.gz to ./comp411/datasets
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


# Part II. Barebones Transformers: Self-Attentional Layer

Here you will complete the implementation of the Pytorch nn.module `SelfAttention`, which will perform the forward pass of a self-attentional layer. Our implementation of the SelfAttentional layer will include three distinct fully connected layers which will be responsible of:

1. A fully connected layer, `W_Q`, which will be used to project our input into `queries`
2. A fully connected layer, `W_K`, which will be used to project our input into `keys`
3. A fully connected layer, `W_V`, which will be used to project our input into `values`

After defining such three fully connected layers, and obtain our `queries, keys, and values` variables at the beginning of our forward pass, the following operations should be carried out in order to complete the attentional layer implementation.

1. Seperate each of `query, key, and value` projections into their respective heads. In other words, split the feature vector dimension of each matrix into necessarry number of chunks.

2. Compute the `Attention Scores` between each pair of sequence elements via conducting a scaled dot product operation between every pair of `queries` and `keys`. Note that `Attention Scores` matrix should have the size of `[# of queries , # of keys]`

3. Calculate the `Attention Weights` of each query by applying the non-linear `Softmax` normalization accross the `keys` dimension of the `Attention Scores` matrix.

4. Obtain the output combination of `values` by matrix multiplying `Attention Weights` with `values`

5. Reassemble heads into one flat vector and return the output.

**HINT**: For a more detailed explanation of the self attentional layer, examine the Appendix A of the original ViT manuscript here:  https://arxiv.org/pdf/2010.11929.pdf 

In [34]:
class SelfAttention(nn.Module):
    
    def __init__(self, input_dims, head_dims=128, num_heads=2,  bias=False):
        super(SelfAttention, self).__init__()
        
        ## initialize module's instance variables
        self.input_dims = input_dims
        self.head_dims = head_dims
        self.num_heads = num_heads
        self.proj_dims = head_dims * num_heads
        
        ## Declare module's parameters
        self.W_Q = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_K = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_V = nn.Linear(input_dims, self.proj_dims,bias=bias)

        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        ## Input of shape, [B, N, D] where:
        ## - B denotes the batch size
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D corresponds to model dimensionality
        b,n,d = x.shape
        
        ## Construct queries,keys,values
        q_ = self.W_Q(x)
        k_ = self.W_K(x)
        v_ = self.W_V(x)
        
        ## Seperate q,k,v into their corresponding heads,
        ## After this operation each q,k,v will have the shape: [B,H,N,D//H] where
        ## - B denotes the batch size
        ## - H denotes number of heads
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D//H corresponds to per head dimensionality
        q, k, v = map(lambda z: torch.reshape(z, (b,n,self.num_heads,self.head_dims)).permute(0,2,1,3), [q_,k_,v_])
       
        #########################################################################################
        # TODO: Complete the forward pass of the SelfAttention layer, follow the comments below #
        #########################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        ## Compute attention logits. Note that this operation is conducted as a
        ## batched matrix multiplication between q and k, the output is scaled by 1/(D//H)^(1/2)
        ## inputs are queries and keys that are both of size [B,H,N,D//H]
        ## Output Attention logits should have the size: [B,H,N,N]
        
        attention_logits = torch.matmul(q, k.permute(0,1,3,2)) / np.sqrt(self.head_dims)
        
        ## Compute attention Weights. Note that this operation is conducted as a
        ## Softmax Normalization across the keys dimension. 
        ## Hint: You can apply the Softmax operation across the final dimension
        
        attention_weights = F.softmax(attention_logits, dim=-1)
        
        ## Compute output values. Note that this operation is conducted as a 
        ## batched matrix multiplication between the Attention Weights matrix and 
        ## the values tensor. After computing output values, the output should be reshaped
        ## Inputs are Attention Weights with size [B, H, N, N], values with size [B, H, N, D//H] = [B, H, N, D//H]
        ## Output should be of size [B, N, D]
        ## Hint: you should use torch.matmul, torch.permute, torch.reshape in that order
        
        attn_out = torch.matmul(attention_weights, v)
        attn_out = torch.permute(attn_out, dims=[0, 2, 1, 3]) #[B, N, H, D//H]
        attn_out = torch.reshape(attn_out, [b, n, -1])

        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             
        ################################################################################
    
        return attn_out

After defining the forward pass of the Self-Attentional Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [37]:
def test_self_attn_layer():
    x = torch.zeros((64, 16, 32), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 256
    layer = SelfAttention(32,64,4)
    out = layer(x)
    print(out.size())  # you should see [64,16,256]
test_self_attn_layer()

torch.Size([64, 16, 256])


# Part III. Barebones Transformers: Transformer Encoder Block

Here you will complete the implementation of the Pytorch nn.module `TransformerBlock`, which will perform the forward pass of a Transfomer Encoder Block. You can refer to Figure 1 of the original manuscript of ViT from this link: https://arxiv.org/pdf/2010.11929.pdf in order to get yourself familiar with the architecture.



In [38]:
## Implementation of a two layer GELU activated Fully Connected Network is provided for you below:

class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, bias=True):
        super().__init__()
        
        self.fc_1 = nn.Linear(input_dims, hidden_dims, bias=bias)
        self.fc_2 = nn.Linear(hidden_dims, output_dims, bias=bias)
        
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)
        
    def forward(self, x):
        o = F.elu(self.fc_1(x))
        o = self.fc_2(o)
        return o

In [39]:
## Build from scratch a TransformerBlock Module. Note that the architecture of this
## module follows a simple computational pipeline:
## input --> layernorm --> SelfAttention --> skip connection 
##       --> layernorm --> MLP ---> skip connection ---> output
## Note that the TransformerBlock module works on a single hidden dimension hidden_dims,
## in order to faciliate skip connections with ease. Be careful about the input arguments
## to the SelfAttention block.


class TransformerBlock(nn.Module):
    def __init__(self, hidden_dims, num_heads=4, bias=False):
        super(TransformerBlock, self).__init__()
        
    ###############################################################
    # TODO: Complete the consturctor of  TransformerBlock module  #
    ###############################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****
    
        self.hidden_dims = hidden_dims
        self.num_heads = num_heads
        
        self.layernorm1 = nn.LayerNorm(hidden_dims)
        self.selfattention = SelfAttention(hidden_dims, head_dims=(hidden_dims//num_heads),num_heads=num_heads, bias=bias)
        self.layernorm2 = nn.LayerNorm(hidden_dims)
        self.mlp = MLP(hidden_dims, hidden_dims, hidden_dims, bias=bias)
        
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ###################################################################
    #                                 END OF YOUR CODE                #             
    ###################################################################
        
    def forward(self, x):
        
    ##############################################################
    # TODO: Complete the forward of TransformerBlock module      #
    ##############################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****
        
        x_norm = self.layernorm1(x)
        attn_out = self.selfattention(x_norm)
        x_skipped = x + attn_out
        x_norm_2 = self.layernorm2(x_skipped)
        mlp = self.mlp(x_norm_2)
        x_skipped_2 = x_skipped + mlp
        return x_skipped_2
        
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ###################################################################
    #                                 END OF YOUR CODE                #             
    ###################################################################

After defining the forward pass of the Transformer Block Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [40]:
def test_transfomerblock_layer():
    x = torch.zeros((64, 16, 128), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 128
    layer = TransformerBlock(128,4) # hidden dims size 128, heads size 4
    out = layer(x)
    print(out.size())  # you should see [64,16,128]
test_transfomerblock_layer()

torch.Size([64, 16, 128])


# Part IV The Vision Transformer (ViT)

The final implementation for the Pytorch nn.module `ViT` is given to you below, which will perform the forward pass of the Vision Transformer. Study it and get yourself familiar with the API.


In [41]:
class ViT(nn.Module):
    def __init__(self, hidden_dims, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4, bias=False):
        super(ViT, self).__init__()
                
        ## initialize module's instance variables
        self.hidden_dims = hidden_dims
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_trans_layers = num_trans_layers
        self.num_heads = num_heads
        self.image_k = image_k
        self.patch_k = patch_k
        
        self.image_height = self.image_width = image_k
        self.patch_height = self.patch_width = patch_k
        
        assert self.image_height % self.patch_height == 0 and self.image_width % self.patch_width == 0,\
                'Image size must be divisible by the patch size.'

        self.num_patches = (self.image_height // self.patch_height) * (self.image_width // self.patch_width)
        self.patch_flat_len = self.patch_height * self.patch_width
        
        ## Declare module's parameters
        
        ## ViT's flattened patch embedding projection:
        self.linear_embed = nn.Linear(self.input_dims*self.patch_flat_len, self.hidden_dims)
        
        ## Learnable positional embeddings, an embedding is learned for each patch location and the class token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, self.hidden_dims))
        
        ## Learnable classt token and its index among attention sequence elements.
        self.cls_token = nn.Parameter(torch.randn(1,1,self.hidden_dims))
        self.cls_index = torch.LongTensor([0])
        
        ## Declare cascaded Transformer blocks:
        transformer_encoder_list = []
        for _ in range(self.num_trans_layers):
            transformer_encoder_list.append(TransformerBlock(self.hidden_dims, self.num_heads, bias))
        self.transformer_encoder = nn.Sequential(*transformer_encoder_list)
        
        ## Declare the output mlp:
        self.out_mlp = MLP(self.hidden_dims, self.hidden_dims, self.output_dims)
         
    def unfold(self, x, f = 7, st = 4, p = 0):
        ## Create sliding window pathes using nn.Functional.unfold
        ## Input dimensions: [B,D,H,W] where
        ## --B : input batch size
        ## --D : input channels
        ## --H, W: input height and width
        ## Output dimensions: [B,N,H*W,D]
        ## --N : number of patches, decided according to sliding window kernel size (f),
        ##      sliding window stride and padding.
        b,d,h,w = x.shape
        x_unf = F.unfold(x, (f,f), stride=st, padding=p)    
        x_unf = torch.reshape(x_unf.permute(0,2,1), (b,-1,d,f*f)).transpose(-1,-2)
        n = x_unf.size(1)
        return x_unf,n
    
    def forward(self, x):
        b = x.size(0)
        ## create sliding window patches from the input image
        x_patches,n = self.unfold(x, self.patch_height, self.patch_height, 0)
        ## flatten each patch into a 1d vector: i.e. 3x4x4 image patch turned into 1x1x48
        x_patch_flat = torch.reshape(x_patches, (b,n,-1))
        ## linearly embed each flattened patch
        x_embed = self.linear_embed(x_patch_flat)
        
        ## retrieve class token 
        cls_tokens = self.cls_token.repeat(b,1,1)
        ## concatanate class token to input patches
        xcls_embed = torch.cat([cls_tokens, x_embed], dim=-2)
        
        ## add positional embedding to input patches + class token 
        xcls_pos_embed = xcls_embed + self.pos_embedding
        
        ## pass through the transformer encoder
        trans_out = self.transformer_encoder(xcls_pos_embed)
        
        ## select the class token 
        out_cls_token = torch.index_select(trans_out, -2, self.cls_index.to(trans_out.device))
        
        ## create output
        out = self.out_mlp(out_cls_token)
        
        return out.squeeze(-2)

After defining the forward pass of the ViT above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [42]:
def test_vit():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size 3,32,32
    model = ViT(hidden_dims=128, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4)
    out = model(x)
    print(out.size())  # you should see [64,10]
test_vit()

torch.Size([64, 10])


# Part V. Train the ViT

### Check Accuracy
Given any minibatch of input data and desired targets, we can check the classification accuracy of a neural network. 

The check_batch_accuracy function is provided for you below:

In [32]:
def check_batch_accuracy(out, target,eps=1e-7):
    b, c = out.shape
    with torch.no_grad():
        _, pred = out.max(-1) 
        correct = np.sum(np.equal(pred.cpu().numpy(), target.cpu().numpy()))
    return correct, np.float(correct) / (b)

### Training Loop
As we have already seen in the Second Assignment, in our PyTorch based training loops, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [33]:
def train(network, optimizer, trainloader):
    """
    Train a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - trainloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall training accuracy for the epoch
    """
    print('\nEpoch: %d' % epoch)
    network.train()  # put model to training mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = Variable(inputs.to(device)), targets.to(device)  # move to device, e.g. GPU
            
        outputs = network(inputs)
        loss =  F.cross_entropy(outputs, targets)
            
        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad() 

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()
            
        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()
            
        loss = loss.detach()
        train_loss += loss.item()
        correct_p, _ = check_batch_accuracy(outputs, targets) 
        correct += correct_p
        total += targets.size(0)

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    return 100.*correct/total

### Evaluation Loop
We have also prepared a Evaluation loop in order to determine our networks capabilities in terms of classification accuracy on a given dataset, either the training, or the validation split

In [14]:
def evaluate(network, evalloader):
    """
    Evaluate a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - evalloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall evaluation accuracy for the epoch
    """
    network.eval() # put model to evaluation mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    eval_loss = 0
    correct = 0
    total = 0
    print('\n---- Evaluation in process ----')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(evalloader):
            inputs, targets = inputs.to(device), targets.to(device) # move to device, e.g. GPU
            outputs = network(inputs)
            loss = F.cross_entropy(outputs, targets)
            
            eval_loss += loss.item()
            correct_p, _ = check_batch_accuracy(outputs, targets)
            correct += correct_p
            total += targets.size(0)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (eval_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total

### Overfit a ViT
Now we are ready to run the training loop. A nice trick is to train your model with just a few training samples in order to see if your implementation is actually bug free. 

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `ViT`. 

You also need to define an optimizer that tracks all the learnable parameters inside `ViT`. We prefer to use `Adam` optimizer for this part.

You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

In [15]:
sample_idx_tr = torch.randperm(len(cifar10_train))[:100]
sample_idx_val = torch.randperm(len(cifar10_train))[-100:]

trainset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_tr)
valset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_val)

print("For overfitting experiments, the subset of the dataset that is used has {} sample images".format(len(trainset_sub)))

batch_size_sub = 25
trainloader_sub = torch.utils.data.DataLoader(trainset_sub, batch_size=batch_size_sub, shuffle=True)
valloader_sub = torch.utils.data.DataLoader(valset_sub, batch_size=batch_size_sub, shuffle=False)

print('==> Data ready, batchsize = {}'.format(batch_size_sub))

For overfitting experiments, the subset of the dataset that is used has 100 sample images
==> Data ready, batchsize = 25


In [17]:
learning_rate = 0.002
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims, input_dims, output_dims, num_trans_layers, num_heads, image_k, patch_k)
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
eval_accs=[]
for epoch in range(10):
    tr_acc = train(network, optimizer, trainloader_sub)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    eval_acc = evaluate(network, valloader_sub)
    print('Evaluation of Epoch {} is completed, Validation accuracy for this epoch is {}'\
              .format(epoch, eval_acc))  
    tr_accs.append(tr_acc)
    eval_accs.append(eval_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final val set accuracy is {}".format(eval_accs[-1]))


Epoch: 0
Loss: 3.074 | Acc: 8.000% (2/25)


C:\Users\Public\anaconda3\envs\comp411\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


Loss: 4.882 | Acc: 10.000% (5/50)
Loss: 4.601 | Acc: 10.667% (8/75)
Loss: 5.011 | Acc: 8.000% (8/100)
Epoch 0 of training is completed, Training accuracy for this epoch is 8.0

---- Evaluation in process ----
Loss: 5.118 | Acc: 16.000% (4/25)
Loss: 4.969 | Acc: 14.000% (7/50)
Loss: 4.993 | Acc: 10.667% (8/75)
Loss: 4.868 | Acc: 13.000% (13/100)
Evaluation of Epoch 0 is completed, Validation accuracy for this epoch is 13.0

Epoch: 1
Loss: 5.322 | Acc: 16.000% (4/25)
Loss: 4.671 | Acc: 20.000% (10/50)
Loss: 4.727 | Acc: 16.000% (12/75)
Loss: 4.384 | Acc: 18.000% (18/100)
Epoch 1 of training is completed, Training accuracy for this epoch is 18.0

---- Evaluation in process ----
Loss: 3.594 | Acc: 16.000% (4/25)
Loss: 3.730 | Acc: 12.000% (6/50)
Loss: 3.523 | Acc: 10.667% (8/75)
Loss: 3.516 | Acc: 10.000% (10/100)
Evaluation of Epoch 1 is completed, Validation accuracy for this epoch is 10.0

Epoch: 2
Loss: 2.489 | Acc: 28.000% (7/25)
Loss: 2.919 | Acc: 20.000% (10/50)
Loss: 2.826 | Acc: 2

## Train the net
By training the four-layer ViT network for three epochs, with untuned hyperparameters that are initialized as below,  you should achieve greater than 50% accuracy both on the training set and the test set:

In [19]:
learning_rate = 0.002
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims, input_dims, output_dims, num_trans_layers, num_heads, image_k, patch_k)
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
test_accs=[]
for epoch in range(3):
    tr_acc = train(network, optimizer, loader_train)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    test_acc = evaluate(network, loader_test)
    print('Evaluation of Epoch {} is completed, Test accuracy for this epoch is {}'\
              .format(epoch, test_acc))  
    
    tr_accs.append(tr_acc)
    test_accs.append(test_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final test set accuracy is {}".format(test_accs[-1]))


Epoch: 0


C:\Users\Public\anaconda3\envs\comp411\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


Loss: 5.264 | Acc: 9.375% (6/64)
Loss: 5.532 | Acc: 9.375% (12/128)
Loss: 5.447 | Acc: 8.333% (16/192)
Loss: 5.144 | Acc: 10.938% (28/256)
Loss: 4.954 | Acc: 10.938% (35/320)
Loss: 4.689 | Acc: 13.281% (51/384)
Loss: 4.505 | Acc: 13.393% (60/448)
Loss: 4.330 | Acc: 13.477% (69/512)
Loss: 4.207 | Acc: 13.021% (75/576)
Loss: 4.078 | Acc: 13.281% (85/640)
Loss: 3.967 | Acc: 13.920% (98/704)
Loss: 3.843 | Acc: 14.844% (114/768)
Loss: 3.771 | Acc: 14.904% (124/832)
Loss: 3.726 | Acc: 15.067% (135/896)
Loss: 3.684 | Acc: 15.104% (145/960)
Loss: 3.611 | Acc: 14.648% (150/1024)
Loss: 3.531 | Acc: 14.982% (163/1088)
Loss: 3.456 | Acc: 15.451% (178/1152)
Loss: 3.410 | Acc: 15.132% (184/1216)
Loss: 3.354 | Acc: 15.312% (196/1280)
Loss: 3.311 | Acc: 15.699% (211/1344)
Loss: 3.271 | Acc: 15.838% (223/1408)
Loss: 3.239 | Acc: 15.897% (234/1472)
Loss: 3.196 | Acc: 16.406% (252/1536)
Loss: 3.168 | Acc: 16.438% (263/1600)
Loss: 3.131 | Acc: 16.887% (281/1664)
Loss: 3.101 | Acc: 16.956% (293/1728)
Loss:

Loss: 2.050 | Acc: 29.541% (4027/13632)
Loss: 2.047 | Acc: 29.644% (4060/13696)
Loss: 2.045 | Acc: 29.680% (4084/13760)
Loss: 2.044 | Acc: 29.731% (4110/13824)
Loss: 2.042 | Acc: 29.788% (4137/13888)
Loss: 2.041 | Acc: 29.824% (4161/13952)
Loss: 2.040 | Acc: 29.852% (4184/14016)
Loss: 2.039 | Acc: 29.865% (4205/14080)
Loss: 2.037 | Acc: 29.900% (4229/14144)
Loss: 2.035 | Acc: 29.934% (4253/14208)
Loss: 2.035 | Acc: 29.933% (4272/14272)
Loss: 2.033 | Acc: 30.015% (4303/14336)
Loss: 2.031 | Acc: 30.028% (4324/14400)
Loss: 2.029 | Acc: 30.088% (4352/14464)
Loss: 2.028 | Acc: 30.094% (4372/14528)
Loss: 2.027 | Acc: 30.140% (4398/14592)
Loss: 2.025 | Acc: 30.151% (4419/14656)
Loss: 2.023 | Acc: 30.183% (4443/14720)
Loss: 2.022 | Acc: 30.249% (4472/14784)
Loss: 2.021 | Acc: 30.260% (4493/14848)
Loss: 2.018 | Acc: 30.291% (4517/14912)
Loss: 2.017 | Acc: 30.315% (4540/14976)
Loss: 2.016 | Acc: 30.326% (4561/15040)
Loss: 2.015 | Acc: 30.369% (4587/15104)
Loss: 2.014 | Acc: 30.380% (4608/15168)


Loss: 1.857 | Acc: 34.566% (9247/26752)
Loss: 1.856 | Acc: 34.588% (9275/26816)
Loss: 1.856 | Acc: 34.587% (9297/26880)
Loss: 1.855 | Acc: 34.601% (9323/26944)
Loss: 1.854 | Acc: 34.623% (9351/27008)
Loss: 1.854 | Acc: 34.652% (9381/27072)
Loss: 1.853 | Acc: 34.681% (9411/27136)
Loss: 1.852 | Acc: 34.695% (9437/27200)
Loss: 1.852 | Acc: 34.712% (9464/27264)
Loss: 1.851 | Acc: 34.734% (9492/27328)
Loss: 1.851 | Acc: 34.733% (9514/27392)
Loss: 1.850 | Acc: 34.765% (9545/27456)
Loss: 1.849 | Acc: 34.775% (9570/27520)
Loss: 1.849 | Acc: 34.781% (9594/27584)
Loss: 1.849 | Acc: 34.795% (9620/27648)
Loss: 1.848 | Acc: 34.822% (9650/27712)
Loss: 1.847 | Acc: 34.847% (9679/27776)
Loss: 1.846 | Acc: 34.881% (9711/27840)
Loss: 1.846 | Acc: 34.898% (9738/27904)
Loss: 1.845 | Acc: 34.911% (9764/27968)
Loss: 1.845 | Acc: 34.907% (9785/28032)
Loss: 1.845 | Acc: 34.909% (9808/28096)
Loss: 1.845 | Acc: 34.922% (9834/28160)
Loss: 1.844 | Acc: 34.931% (9859/28224)
Loss: 1.844 | Acc: 34.930% (9881/28288)


...

Loss: 1.436 | Acc: 48.047% (4797/9984)
Loss: 1.435 | Acc: 48.040% (4804/10000)
Evaluation of Epoch 1 is completed, Test accuracy for this epoch is 48.04

Epoch: 2
Loss: 1.552 | Acc: 48.438% (31/64)
Loss: 1.494 | Acc: 47.656% (61/128)
Loss: 1.486 | Acc: 47.396% (91/192)
Loss: 1.444 | Acc: 47.656% (122/256)
Loss: 1.422 | Acc: 47.812% (153/320)
Loss: 1.394 | Acc: 48.438% (186/384)
Loss: 1.415 | Acc: 47.545% (213/448)
Loss: 1.403 | Acc: 48.047% (246/512)
Loss: 1.433 | Acc: 47.222% (272/576)
Loss: 1.418 | Acc: 47.500% (304/640)
Loss: 1.402 | Acc: 47.869% (337/704)
Loss: 1.388 | Acc: 48.177% (370/768)
Loss: 1.408 | Acc: 47.716% (397/832)
Loss: 1.413 | Acc: 47.991% (430/896)
Loss: 1.395 | Acc: 48.646% (467/960)
Loss: 1.400 | Acc: 48.340% (495/1024)
Loss: 1.390 | Acc: 48.713% (530/1088)
Loss: 1.397 | Acc: 48.351% (557/1152)
Loss: 1.407 | Acc: 47.944% (583/1216)
Loss: 1.403 | Acc: 48.203% (617/1280)
Loss: 1.400 | Acc: 48.661% (654/1344)
Loss: 1.395 | Acc: 48.793% (687/1408)
Loss: 1.387 | Acc: 4

Loss: 1.356 | Acc: 50.823% (6733/13248)
Loss: 1.356 | Acc: 50.811% (6764/13312)
Loss: 1.356 | Acc: 50.785% (6793/13376)
Loss: 1.356 | Acc: 50.774% (6824/13440)
Loss: 1.356 | Acc: 50.733% (6851/13504)
Loss: 1.356 | Acc: 50.693% (6878/13568)
Loss: 1.356 | Acc: 50.682% (6909/13632)
Loss: 1.357 | Acc: 50.679% (6941/13696)
Loss: 1.356 | Acc: 50.734% (6981/13760)
Loss: 1.358 | Acc: 50.716% (7011/13824)
Loss: 1.357 | Acc: 50.713% (7043/13888)
Loss: 1.357 | Acc: 50.731% (7078/13952)
Loss: 1.358 | Acc: 50.742% (7112/14016)
Loss: 1.357 | Acc: 50.760% (7147/14080)
Loss: 1.358 | Acc: 50.735% (7176/14144)
Loss: 1.358 | Acc: 50.753% (7211/14208)
Loss: 1.358 | Acc: 50.750% (7243/14272)
Loss: 1.359 | Acc: 50.774% (7279/14336)
Loss: 1.359 | Acc: 50.785% (7313/14400)
Loss: 1.358 | Acc: 50.823% (7351/14464)
Loss: 1.358 | Acc: 50.840% (7386/14528)
Loss: 1.359 | Acc: 50.761% (7407/14592)
Loss: 1.359 | Acc: 50.744% (7437/14656)
Loss: 1.359 | Acc: 50.768% (7473/14720)
Loss: 1.359 | Acc: 50.785% (7508/14784)


Loss: 1.354 | Acc: 50.907% (13358/26240)
Loss: 1.354 | Acc: 50.863% (13379/26304)
Loss: 1.354 | Acc: 50.891% (13419/26368)
Loss: 1.354 | Acc: 50.874% (13447/26432)
Loss: 1.354 | Acc: 50.891% (13484/26496)
Loss: 1.354 | Acc: 50.885% (13515/26560)
Loss: 1.354 | Acc: 50.886% (13548/26624)
Loss: 1.354 | Acc: 50.854% (13572/26688)
Loss: 1.353 | Acc: 50.860% (13606/26752)
Loss: 1.353 | Acc: 50.884% (13645/26816)
Loss: 1.353 | Acc: 50.893% (13680/26880)
Loss: 1.353 | Acc: 50.909% (13717/26944)
Loss: 1.353 | Acc: 50.915% (13751/27008)
Loss: 1.353 | Acc: 50.894% (13778/27072)
Loss: 1.354 | Acc: 50.859% (13801/27136)
Loss: 1.354 | Acc: 50.882% (13840/27200)
Loss: 1.353 | Acc: 50.899% (13877/27264)
Loss: 1.353 | Acc: 50.893% (13908/27328)
Loss: 1.353 | Acc: 50.898% (13942/27392)
Loss: 1.353 | Acc: 50.889% (13972/27456)
Loss: 1.353 | Acc: 50.890% (14005/27520)
Loss: 1.353 | Acc: 50.910% (14043/27584)
Loss: 1.353 | Acc: 50.930% (14081/27648)
Loss: 1.353 | Acc: 50.924% (14112/27712)
Loss: 1.353 | Ac

Loss: 1.355 | Acc: 50.827% (19843/39040)
Loss: 1.355 | Acc: 50.836% (19879/39104)
Loss: 1.355 | Acc: 50.817% (19904/39168)
Loss: 1.356 | Acc: 50.800% (19930/39232)
Loss: 1.356 | Acc: 50.809% (19966/39296)
Loss: 1.356 | Acc: 50.805% (19997/39360)
Loss: 1.355 | Acc: 50.819% (20035/39424)
Loss: 1.356 | Acc: 50.815% (20066/39488)
Loss: 1.356 | Acc: 50.807% (20095/39552)
Loss: 1.356 | Acc: 50.800% (20125/39616)
Loss: 1.356 | Acc: 50.806% (20160/39680)
Loss: 1.356 | Acc: 50.805% (20192/39744)
Loss: 1.356 | Acc: 50.816% (20229/39808)
Loss: 1.356 | Acc: 50.800% (20255/39872)
Loss: 1.356 | Acc: 50.799% (20287/39936)
Loss: 1.356 | Acc: 50.790% (20316/40000)
Loss: 1.356 | Acc: 50.801% (20353/40064)
Loss: 1.356 | Acc: 50.817% (20392/40128)
Loss: 1.356 | Acc: 50.816% (20424/40192)
Loss: 1.356 | Acc: 50.797% (20449/40256)
Loss: 1.356 | Acc: 50.794% (20480/40320)
Loss: 1.356 | Acc: 50.805% (20517/40384)
Loss: 1.357 | Acc: 50.799% (20547/40448)
Loss: 1.357 | Acc: 50.795% (20578/40512)
Loss: 1.357 | Ac

Loss: 1.322 | Acc: 51.111% (1472/2880)
Loss: 1.320 | Acc: 51.257% (1509/2944)
Loss: 1.321 | Acc: 51.363% (1545/3008)
Loss: 1.323 | Acc: 51.432% (1580/3072)
Loss: 1.322 | Acc: 51.339% (1610/3136)
Loss: 1.321 | Acc: 51.312% (1642/3200)
Loss: 1.321 | Acc: 51.225% (1672/3264)
Loss: 1.321 | Acc: 51.202% (1704/3328)
Loss: 1.320 | Acc: 51.238% (1738/3392)
Loss: 1.320 | Acc: 51.418% (1777/3456)
Loss: 1.318 | Acc: 51.477% (1812/3520)
Loss: 1.318 | Acc: 51.535% (1847/3584)
Loss: 1.320 | Acc: 51.343% (1873/3648)
Loss: 1.319 | Acc: 51.428% (1909/3712)
Loss: 1.319 | Acc: 51.351% (1939/3776)
Loss: 1.316 | Acc: 51.536% (1979/3840)
Loss: 1.318 | Acc: 51.434% (2008/3904)
Loss: 1.318 | Acc: 51.462% (2042/3968)
Loss: 1.318 | Acc: 51.463% (2075/4032)
Loss: 1.321 | Acc: 51.367% (2104/4096)
Loss: 1.323 | Acc: 51.346% (2136/4160)
Loss: 1.322 | Acc: 51.373% (2170/4224)
Loss: 1.321 | Acc: 51.376% (2203/4288)
Loss: 1.320 | Acc: 51.425% (2238/4352)
Loss: 1.318 | Acc: 51.472% (2273/4416)
Loss: 1.316 | Acc: 51.540